This notebook is created to test the configuration settings required to connect Microsoft SQL Server Management Srvices (SSMS) 2016. Data is loaded to db and retrived using different connection settings. We will be using pandas to handle data in python.

## 1. Connect to SQL Server & read data to pandas dataframe

- [Using: pymssql](#Using:-pymssql)
- [Using: pyodbc](#Using:-pyodbc)
- [Using: sqlalchemy](#Using:-sqlalchemy)

### Using: pymssql

Connection to SQL Server is done using Sql Sqever Admin/User Credentials. 

In [1]:
import pymssql
import pandas
import io
import json

In [2]:
# Load username and password from json document
with io.open('login_credentials.json') as cred:
        creds = json.load(cred)

In [3]:
# Create the connection
conn = pymssql.connect(server='WIN-H12AB3456H\SQL2016STD', 
                       user=creds['UID'], 
                       password=creds['PWD'], 
                       database='DemoDB01')

In [4]:
# Database Query
sql = """

SELECT * 
  FROM dbo.student

"""

# Read data from db to Pandas
df = pandas.io.sql.read_sql(sql, conn)
df

,ID,firstname,lastname,age,color
0,1,Alice,M,20,Red
1,2,Robb,K,26,Blue
2,3,Adam,M,29,Red
3,4,Roy,G,40,Green


### Using: pyodbc

Connection to SQL Server is done using Windows Authentication 

(To be able to run the below query, you must have Windows authentication enabled for SQL Server 'SQL2016STD')

In [5]:
import pyodbc

In [6]:
# Parameters
conn_str = (
    r'Driver={SQL Server};'
    r'Server=.\SQL2016STD;'
    r'Database=DemoDB01;'
    r'Trusted_Connection=yes;'
    )

# Create the connection
cnxn = pyodbc.connect(conn_str)

In [7]:
# Database Query
sql = """
SELECT * 
  FROM dbo.student
"""

# Read data from db to Pandas
df = pandas.io.sql.read_sql(sql, cnxn)
df

,ID,firstname,lastname,age,color
0,1,Alice,M,20,Red
1,2,Robb,K,26,Blue
2,3,Adam,M,29,Red
3,4,Roy,G,40,Green


### Using: sqlalchemy

Connection to SQL Server is done using Windows Authentication 

(To be able to run the below query, you must have Windows authentication enabled for SQL Server 'SQL2016STD')

In [8]:
from sqlalchemy import create_engine

In [9]:
# Parameters
Driver="driver=SQL Server"
Server=".\SQL2016STD"
Database="DemoDB01" 

# Create the connection
engine = create_engine('mssql+pyodbc://' + Server + '/' + Database + "?" + Driver)

# Database Query
sql = """
SELECT * 
  FROM dbo.student
"""

# Read data from db to Pandas
df = pandas.read_sql_query(sql, engine)
df

,ID,firstname,lastname,age,color
0,1,Alice,M,20,Red
1,2,Robb,K,26,Blue
2,3,Adam,M,29,Red
3,4,Roy,G,40,Green


### Load data into SQL Server

In [10]:
df['ID'] = df['ID'] + 4

In [11]:
df

,ID,firstname,lastname,age,color
0,5,Alice,M,20,Red
1,6,Robb,K,26,Blue
2,7,Adam,M,29,Red
3,8,Roy,G,40,Green


In [12]:
# Parameters
Driver="driver=SQL Server"
Server=".\SQL2016STD"
Database="DemoDB01" 
table = 'student'
engine = engine = create_engine('mssql+pyodbc://'+ Server + '/' + Database + "?" + Driver)

df.to_sql(table, engine, if_exists='append', index=False)

In [13]:
inserted = pandas.read_sql('SELECT * from %s' %(table), engine)

In [14]:
inserted

,ID,firstname,lastname,age,color
0,1,Alice,M,20,Red
1,2,Robb,K,26,Blue
2,3,Adam,M,29,Red
3,4,Roy,G,40,Green
4,5,Alice,M,20,Red
5,6,Robb,K,26,Blue
6,7,Adam,M,29,Red
7,8,Roy,G,40,Green


## References:

1. http://stackoverflow.com/questions/16515420/connecting-to-ms-sql-server-with-windows-authentication-using-python

2. http://nbviewer.jupyter.org/urls/bitbucket.org/hrojas/learn-pandas/raw/master/lessons/08%20-%20Lesson.ipynb